# BTTH05: Softmax Regression

TODO: Ghi họ tên và MSSV của bạn (vd, Nguyễn Văn A - 1234567)

---



## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Lưu ý: tuyệt đối không gian lận. Nếu vi phạm thì bạn sẽ bị 0 điểm cho cả phần thực hành môn học. Nên nhớ mục tiêu chính ở đây là học kiến thức.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Cell` - `Run All` để chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Cell` - `Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt file `Ex05-SoftmaxRegression.ipynb` (không cần nộp file dữ liệu `mnist.pkl.gz`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [5]:
%matplotlib inline
import numpy as np
#import matplotlib.pyplot as plt
import cPickle
import gzip
import random
# You can also import other things ...

## 3. Hàm đọc dữ liệu

Trong bài này, bạn sẽ thử nghiệm Softmax Regression trên bộ dữ liệu MNIST (file `mnist.pkl.gz` đính kèm). Đây là bộ dữ liệu gồm các ảnh chữ số viết tay từ 0-9 (10 lớp); mỗi ảnh có kích thước $28\times 28$ và là ảnh grayscale. Bộ dữ liệu đã được chia sẵn làm 3 tập: tập huấn luyện gồm 50000 ảnh, tập validation gồm 10000 ảnh (tạm thời mình sẽ không dùng đến tập này), và tập test gồm 10000 ảnh.

In [6]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.plk.gz').
    
    Returns
    -------
    (train_X, train_Y, val_X, val_Y, test_X, test_Y) : tuple
        train_X : numpy array, shape (N=50000, d+1=785)
            Input vectors of the training set.
        train_Y: numpy array, shape (N=50000)
            Outputs of the training set.
        val_X : numpy array, shape (N=10000, d+1=785)
            Input vectors of the validation set.
        val_Y: numpy array, shape (N=10000)
            Outputs of the validation set.
        test_X : numpy array, shape (N=10000, d+1=785)
            Input vectors of the test set.
        test_Y: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open('mnist.pkl.gz')
    train_data, val_data, test_data = cPickle.load(f)
    f.close()
    train_X , train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data
    return (train_X, train_Y, val_X, val_Y, test_X, test_Y)
    # TODO

## 4. Hàm huấn luyện Softmax Regression

In [7]:
def compute_softmax_output(X, W):
    """
    Computes the outputs of Softmax Regression.
    
    Parameters
    ----------
    X : numpy array, shape (N, d+1)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones (corresponding to x_0).
    W : numpy array, shape(d+1, K=10)
        The matrix of Softmax's parameters; each column corresponds to parameters of a class.
    
    Returns
    -------
    A : numpy array, shape (N, K=10)
        The maxtrix of Softmax's output vectors; each row is an output vector (containing each class's 
        probability given the corresponding input vector).
    """
    Z = X.dot(W)
    A = np.exp(Z)
    A /= np.sum(A, axis = 1, keepdims=True)
    return A
    # TODO

def train_softmax(X, Y, learning_rate, mnb_size, max_epoch):
    """
    Trains Softmax Regression on the dataset (X, Y).
    Cost function: mean negative log likelihood (it's the one I talked in the class).
    Optimization algorithm: Stochastic Gradient Descent (SGD).
    
    Your code need to print out the Mean Binary Error on the training set after each epoch
    (e.g., 'Epoch ..., training err ...%').
    
    Parameters
    ----------
    X : numpy array, shape (N, d + 1)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones (corresponding to x_0).
    Y : numpy array, shape (N)
        The vector of outputs.
    learning_rate : float
        Learning rate of SGD.
    mnb_size : int
        Minibatch size of SGD.
    max_epoch : int
        After this number of epochs, we'll terminate SGD.
    
    Returns
    -------
    W : numpy array, shape (d+1, K=10)
        Parameters of Softmax Regression after training.         
    """
    W = np.zeros((X.shape[1],10))
    N = X.shape[0]
    one_hot_Y = np.zeros((N,10))
    for i in range(N):
        one_hot_Y[i][Y[i]] = 1
    rand_idxs = range(N)
    for epoch in range(max_epoch):
        np.random.shuffle(rand_idxs)
        for start_idx in range(0, N, mnb_size):
            mnb_X = X[rand_idxs[start_idx:start_idx + mnb_size]]
            mnb_Y = one_hot_Y[rand_idxs[start_idx:start_idx + mnb_size]]
            A = compute_softmax_output(mnb_X, W)
            grad = mnb_X.T.dot(A - mnb_Y) / mnb_size
            W = W - learning_rate * grad
    return W
    # TODO

## 5. Chạy

Bây giờ, bạn sẽ dùng các hàm đã định nghĩa ở trên như sau:

1. Đọc dữ liệu
2. Huấn luyện Softmax Regression trên tập huấn luyện. Chạy SGD với `learning_rate = 0.03`, `mnb_size = 10`, và `max_epoch = 20`.
3. Đánh giá bộ phân lớp học được bằng cách đo độ lỗi Mean Binary Error trên tập kiểm tra.

In [8]:
# TODO
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.plk.gz')
W = train_softmax(train_X, train_Y, 0.03, 10, 20)
A = compute_softmax_output(test_X, W)
num = 0
for i in range(test_X.shape[0]):
    idx = A[i].argmax(axis=0)
    if (idx != test_Y[i]):
        num = num + 1
print (num*1.0) / test_X.shape[0]

0.0765
